# Portugese Meals Classification: Training

In [1]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from IPython.display import clear_output
import yaml

In [2]:
%cd ..

d:\GigaFolder\projects\Portuguese-Meals-Classification


In [3]:
config = yaml.safe_load(open('config.yaml', 'r'))

In [4]:
def uncompress_splits(dir: str):
    X = np.load(dir + 'Xvalues.npz')['arr_0']
    Y = np.load(dir + 'Yvalues.npz')['arr_0']

    return X, Y

In [6]:
images, labels = uncompress_splits(config['dataset']['augmented_dir'])
assert len(images) == len(labels)

In [23]:
x_train, x_test, y_train, y_test = train_test_split(
    images, labels, train_size=config['dataset']['train_ratio'], random_state=config['random_seed'], shuffle=True, stratify=labels)
x_val, x_test, y_val, y_test = train_test_split(
    images, labels, train_size=config['dataset']['validation_ratio'], random_state=config['random_seed'], shuffle=True, stratify=labels)


In [ ]:
import tensorflow as tf
from keras import regularizers
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.optimizers import Adamax
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt


In [ ]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2M(
    include_top=False, weights="imagenet", input_shape=config['img_shape'], pooling='max')

base_model.trainable = config['train']['base_trainable']

x = base_model.output
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
x = Dense(256, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu')(x)
x = Dropout(rate=.2, seed=config['random_seed'])(x)
output = Dense(config['class_count'], activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(Adamax(learning_rate=config['train']['lr']),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=config['train']['label_smoothing']), metrics=config['train']['metrics'])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import wandb
from sklearn.preprocessing import LabelBinarizer


In [ ]:
# I need to reverse one-hot encoded labels
# I need to reverse one-hot encoded labels
classes = ['aletria',
           'arroz_cabidela',
           'bacalhau_bras',
           'bacalhau_natas',
           'batatas_fritas',
           'bolo_chocolate',
           'cachorro',
           'caldo_verde',
           'cozido_portuguesa',
           'croissant',
           'donuts',
           'esparguete_bolonhesa',
           'feijoada',
           'francesinha',
           'gelado',
           'hamburguer',
           'jardineira',
           'nata',
           'ovo',
           'pasteis_bacalhau',
           'pizza',
           'tripas_moda_porto',
           'waffles']

classes.sort()

encoder = LabelBinarizer()
classes_oh = np.asarray(classes)
classes_oh = encoder.fit_transform(classes_oh)

In [ ]:
def resize(image, image_size=config['img_shape'][0]):
    # I'll go for a square image
    return cv2.resize(image, (image_size, image_size), interpolation=cv2.INTER_AREA)

def normalize(image):
    image = image / 255
    return image

def preprocess(image):
    image = resize(image)
    image = normalize(image)
    return image

In [ ]:
def validate_on_dataframe(model, x, y, name):

    clss = encoder.inverse_transform(np.array(y))
    preds = model.predict(x)

    res = []
    for idx in range(len(preds)):
        res.append(np.argmax(y[idx]) == np.argmax(preds[idx]))
        # print(f'True idx = {np.argmax(true[idx])}')
        # print(f'Predicted idx = {np.argmax(preds[idx])}')
    print(f'Accuracy: {np.sum(res) / np.sum(y)*100:.04}%')

    preds = encoder.inverse_transform(np.array(preds))
    cm = confusion_matrix(np.array(classes), preds)
    plt.figure(figsize=(20, 16))
    cm = sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)
    plt.xticks(np.arange(config['class_count'])+.5, clss, rotation=90)
    plt.yticks(np.arange(config['class_count'])+.5, clss, rotation=0)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()
    # create classification report
    clr = classification_report(clss, preds, target_names=classes, digits=4)
    print(f"Classification Report for {name}:\n----------------------\n", clr)

    wandb.log({f'confusion_matrix_{name}': wandb.Image(cm)})
    wandb.log({f'{name}_accuracy:': np.sum(res) / np.sum(y)*100})


In [ ]:
with wandb.init(project=config['wandb']['project'],
                name=config['wandb']['name'],
                job_type='Training+Validation',
                config=config):
    callbacks = [EarlyStopping(**config['callbacks']['EarlyStopping']),
                 ReduceLROnPlateau(**config['callbacks']['ReduceLROnPlateau']),
                 WandbCallback(**config['callbacks']['WandbCallback'])]

    history = model.fit(
        x_train, y_train,
        epochs=config['train']['epochs'],
        batch_size=config['train']['batch_size'],
        validation_data=(x_test, y_test),
        callbacks=callbacks
    )
    validate_on_dataframe(model, x_val, y_val, 'baseline')

In [ ]:
model.save(config['baseline_path'])

In [ ]:
tf.saved_model.save(model, config['saved_model_baseline_path'])